In [330]:
import numpy as np 
import pandas as pd 
import portion as P
import itertools
from scipy.optimize import linprog
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable 
from math import prod


# Transforming LCL:s to linear problems

Goal is to find $\alpha, \beta, a,b,c...$ such that if some $A, A, B$ is possible for active/passive nodes, then the corresponding $a+a+b \leq \beta / \geq \alpha$. Additionally it would be great to see if these values $a, b, c...$ could be expanded to intervals.

Example: problem

```
A AB AB

B AB AB
```
can be interpreted as $\alpha = 1, \beta=2, A = \left[ 0,\frac{1}{3}\right), B =  \left(\frac{2}{3}, 1\right]$.

What is the system of linear inequalities that could imply this?

\begin{align*}
3a &\leq \beta \\
2a+b &\leq \beta \\
a+2b &\leq \beta \\
3b &> \beta \\
\\
3a &< \alpha \\
2a+b &\geq \alpha \\
a+2b &\geq \alpha \\
3b &\geq \alpha

\end{align*}

In [331]:
# Initialize variables

# Add active and passive constraints in RE-formalism
active = """
A A A
A A B
A A C
A B B
"""
passive = """
A A C
A B C
A C C
B B B
B B C
B C C
C C C
"""

# Enable/disable debug mode
debug = True

In [332]:
actives = list(map(lambda x: x.split(" "), active.strip().split("\n")))
passives = list(map(lambda x: x.split(" "), passive.strip().split("\n")))

variables = sorted(list(set(list("".join("".join([active, passive]).split())))))

combinations = pd.DataFrame({"combination": itertools.combinations_with_replacement(variables, len(passives[0]))})
combinations["active"] = False
combinations["passive"] = False

for row in passives:
    for c in map(lambda c: tuple(sorted(c)), itertools.product(*row)):
        combinations.loc[combinations["combination"]==c, "passive"] = True 

for row in actives:
    for c in map(lambda c: tuple(sorted(c)), itertools.product(*row)):
        combinations.loc[combinations["combination"]==c, "active"] = True 

if debug: print(combinations)


  combination  active  passive
0   (A, A, A)    True    False
1   (A, A, B)    True    False
2   (A, A, C)    True     True
3   (A, B, B)    True    False
4   (A, B, C)   False     True
5   (A, C, C)   False     True
6   (B, B, B)   False     True
7   (B, B, C)   False     True
8   (B, C, C)   False     True
9   (C, C, C)   False     True


In [333]:
epsilon = 0.001


model = LpProblem(name="Reductions", sense=LpMaximize)

alpha = LpVariable(name = "alpha_0", lowBound=0, upBound=10)

alpha = LpVariable(name = "alpha_1", lowBound=0, upBound=10)

pulp_variables = dict(zip(variables, [LpVariable(name = v, lowBound=0, upBound=1) for v in variables]))

# Try different targets in order to get some readable solutions
model += beta-alpha

# Add initial constraints
#for v in pulp_variables.values():
#    model += (v>=0, f"{v}, low")
#    model += (v<=1, f"{v}, high")

# Add constraints

for index, row in combinations.iterrows():
    if row["active"]:
        model += (sum(pulp_variables[c] for c in row["combination"])>=alpha, f"{index}_Active")
    else:
        model += (sum(pulp_variables[c] for c in row["combination"])<= alpha-epsilon, f"{index}_not_Active")

    if row["passive"]:
        model += (sum(pulp_variables[c] for c in row["combination"])<=beta, f"{index}_Passive")
    else:
        model += (sum(pulp_variables[c] for c in row["combination"])>=beta+epsilon, f"{index}_not_Passive")
    

    
if debug: print(model)

if model.solve() == 1:
    print("Linear model found:")

    for var in model.variables():
        print(var, var.value())

Reductions:
MAXIMIZE
0*alpha_1 + 0*beta + 0
SUBJECT TO
0_Active: 3 A - alpha_1 >= 0

0_not_Passive: 3 A - beta >= 0.001

1_Active: 2 A + B - alpha_1 >= 0

1_not_Passive: 2 A + B - beta >= 0.001

2_Active: 2 A + C - alpha_1 >= 0

2_Passive: 2 A + C - beta <= 0

3_Active: A + 2 B - alpha_1 >= 0

3_not_Passive: A + 2 B - beta >= 0.001

4_not_Active: A + B + C - alpha_1 <= -0.001

4_Passive: A + B + C - beta <= 0

5_not_Active: A + 2 C - alpha_1 <= -0.001

5_Passive: A + 2 C - beta <= 0

6_not_Active: 3 B - alpha_1 <= -0.001

6_Passive: 3 B - beta <= 0

7_not_Active: 2 B + C - alpha_1 <= -0.001

7_Passive: 2 B + C - beta <= 0

8_not_Active: B + 2 C - alpha_1 <= -0.001

8_Passive: B + 2 C - beta <= 0

9_not_Active: 3 C - alpha_1 <= -0.001

9_Passive: 3 C - beta <= 0

VARIABLES
A <= 1 Continuous
B <= 1 Continuous
C <= 1 Continuous
alpha_1 <= 10 Continuous
beta <= 10 Continuous

Linear model found:
A 0.005
B 0.003
C 0.0
alpha_1 0.01
beta 0.01
